### 문제

* 각 세대별 자식이 없는 개체의 수(COUNT)와 세대(GENERATION)를 출력하는 SQL문을 작성해주세요. 이때 결과는 세대에 대해 오름차순 정렬해주세요. 단, 모든 세대에는 자식이 없는 개체가 적어도 1개체는 존재합니다.

__MYSQL__

```SQL

WITH 
    G1 AS (
        SELECT
            ID
        FROM
            ECOLI_DATA
        WHERE
            PARENT_ID IS NULL
),
    G2 AS (
        SELECT
            ECOLI_DATA.ID
        FROM
            ECOLI_DATA
        JOIN G1
            ON G1.ID = ECOLI_DATA.PARENT_ID
),
    G3 AS (
        SELECT
            ECOLI_DATA.ID
        FROM
            ECOLI_DATA
        JOIN G2
            ON G2.ID = ECOLI_DATA.PARENT_ID
),
    G4 AS (
        SELECT
            ECOLI_DATA.ID
        FROM
            ECOLI_DATA
        JOIN G3
            ON G3.ID = ECOLI_DATA.PARENT_ID
),
    COMBINED AS (
        SELECT
            T1.ID,
            CASE
                WHEN T1.ID = G1.ID THEN 1
                WHEN T1.ID = G2.ID THEN 2
                WHEN T1.ID = G3.ID THEN 3
                WHEN T1.ID = G4.ID THEN 4
            END AS GENERATION
        FROM 
            ECOLI_DATA T1
        LEFT JOIN G1
            ON T1.ID = G1.ID
        LEFT JOIN G2
            ON T1.ID = G2.ID
        LEFT JOIN G3
            ON T1.ID = G3.ID
        LEFT JOIN G4
            ON T1.ID = G4.ID
        GROUP BY
            T1.ID
)

SELECT
    COUNT(T2.ID) AS COUNT,
    T3.GENERATION
FROM
    ECOLI_DATA T2
JOIN COMBINED T3
    ON T2.ID = T3.ID
WHERE 
    T2.ID NOT IN (
                SELECT DISTINCT 
                    PARENT_ID 
                FROM 
                    ECOLI_DATA
                WHERE
                    PARENT_ID IS NOT NULL
    )
GROUP BY
    GENERATION
ORDER BY
    GENERATION ASC;

```


__수정__

```SQL 

WITH RECURSIVE ECOLI_TREE AS (
    SELECT
        ID,
        PARENT_ID,
        1 AS GENERATION
    FROM
        ECOLI_DATA
    WHERE
        PARENT_ID IS NULL

    UNION ALL

    SELECT
        E.ID,
        E.PARENT_ID,
        T.GENERATION + 1 AS GENERATION
    FROM
        ECOLI_DATA E
    JOIN
        ECOLI_TREE T
    ON
        E.PARENT_ID = T.ID
)

SELECT
    COUNT(T.ID) AS COUNT,
    T.GENERATION
FROM
    ECOLI_TREE T
LEFT JOIN
    ECOLI_DATA E
ON
    T.ID = E.PARENT_ID
WHERE
    E.ID IS NULL
GROUP BY
    T.GENERATION
ORDER BY
    T.GENERATION ASC;

```